# Requirements

In [2]:
from ipywidgets import interact_manual
from vpython import canvas, helix, rate, scene, sphere, vector

<IPython.core.display.Javascript object>

# Problem setting

We consider a mass suspended from a spring.  The mass $m$ as well as the spring constant $k$ and a length in rest of $l_0$ can be specified. The other end of the spring is suspended from a fixed point at the origin $(0, 0, 0)$.

Gravity acts on the mass, while we assume that the spring is massless.  The gravitational force is $F_g = -m g$ where $g = 9.81 \frac{m}{s^2}$ is the gravitational acceleration on Earth.  This force acts along the $Z$-axis.

The force exerted by the spring on the sphere when it has length $l$ is given by Hooke's law, $F_s = k (l - l_0)$.  This force also acts along the $Z$-axis.

At time $t = 0$, the length of the spring is $l_0$, so the position of the mass will be $(0, 0, -l_0)$.  The mass is also assumed to be at rest.

# Implementation

We define the simulation of the system as a function that takes three arguments:
* the maximum time $t_{\rm{max}}$ we want to run the simulation for;
* the mass $m$; and
* the spring constant $k$.

We assume that the spring length in rest is $l_0 = 1$.

We will compute the position of the mass at intervals of $\Delta t = 0.01$ using Euler's integration method and Newton's second law of motion.

$$
    \begin{array}{rcl}
        m \frac{d\vec{v}}{dt} & = & \vec{F} \\
        \frac{d\vec{x}}{dt} & = & \vec{v} \\
    \end{array}
$$

Here, $\vec{x}$ and $\vec{v}$ are the position and velocity of the mass, and $\vec{F}$ is the total force acting on the mass.

These equation can be discretized as follows:

$$
    \begin{array}{rcl}
        \vec{v}_{t + \Delta t} & = & \vec{v}_t + \frac{\vec{F}}{m} \Delta t \\
        \vec{x}_{t + \Delta t} & = & \vec{x}_t + \vec{v} \Delta t
    \end{array}
$$

We use vpython to visualize the movement of the spring, and ipywidgets to create the controls for the simulation.

In [4]:
@interact_manual(t=(1, 20), m=(0.1, 10), k=(0.1, 20))
def animate_spring(t_max=10.0, m=1.0, k=9.81):
    # Define the constants.
    l_0 = 1.0
    g = 9.81
    
    # Create a canvas, making sure we look at the system from the
    # proper direction.
    canvas(forward=vector(0.0, -1.0, 0.0))
    
    # Define the position of the fixed point and the initial
    # position of the mass.
    fixed_pos = vector(0.0, 0.0, 0.5)
    mass_pos = vector(0.0, 0.0, -l_0)
    
    # Define the vector that initially connects the pivot point to
    # the mass.
    init_spring_axis = mass_pos - fixed_pos
    
    # Visualize the spring as a helix, finetuning its appearance
    # by specifying the radius and the width.
    spring = helix(pos=fixed_pos, axis=init_spring_axis,
                   radius=0.2, width=0.1)
    
    # Visualize the fixed point as well as the mass as spheres
    # the radius of the sphere for the mass depends on m
    # through the constant density (1.0 for convenience)
    # and the volume of the mass.
    fixed = sphere(pos=fixed_pos, radius=0.1)
    mass = sphere(pos=mass_pos, radius=0.2*m**(1/3))
    
    # Set the initial velocity of the mass to 0.
    mass.v = vector(0.0, 0.0, 0.0)
    
    # The simulation starts at t = 0, and delta_t = 0.01
    t = 0.0
    delta_t = 0.01
    while t <= t_max:
        # We set the rate for the visualization at 1/delta_t.
        rate(int(1.0/delta_t))
        # Compute the forces that act on the mass
        grav_force = m*g*vector(0.0, 0.0, -1.0)
        spring_force = k*(init_spring_axis - spring.axis)
        # Update the velocity of the mass according to Newton's la.
        mass.v += delta_t*(grav_force + spring_force)/m
        # Compute the new position of the mass, given its velocity.
        mass.pos += delta_t*mass.v
        # Update the axis of the spring based on the new position of
        # the mass.
        spring.axis = mass.pos - fixed.pos
        # Update the current time.
        t += delta_t

interactive(children=(FloatSlider(value=10.0, description='t_max', max=30.0, min=-10.0), FloatSlider(value=1.0…